In [1]:
# Import dependencies
import pandas as pd
from sqlalchemy import create_engine

# Import config file so as not to push password and/or username
from config import (user, password, host, port, database)

# Extract CSVs into DataFrames

In [2]:
# Read adult csv
adult_file = "Resources/adult.csv"
adult_df = pd.read_csv(adult_file)
adult_df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [3]:
# Read country csv and replace the ',' in the values to decimals
country_file = "Resources/countries_of_the_world.csv"
country_df = pd.read_csv(country_file, decimal=",")
country_df.head()

,Country,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,647500,48.0,0.00,23.06,163.07,700.0,36.0,3.2,12.13,0.22,87.65,1.0,46.60,20.34,0.380,0.240,0.380
1,Albania,EASTERN EUROPE,3581655,28748,124.6,1.26,-4.93,21.52,4500.0,86.5,71.2,21.09,4.42,74.49,3.0,15.11,5.22,0.232,0.188,0.579
2,Algeria,NORTHERN AFRICA,32930091,2381740,13.8,0.04,-0.39,31.00,6000.0,70.0,78.1,3.22,0.25,96.53,1.0,17.14,4.61,0.101,0.600,0.298
3,American Samoa,OCEANIA,57794,199,290.4,58.29,-20.71,9.27,8000.0,97.0,259.5,10.00,15.00,75.00,2.0,22.46,3.27,NaN,NaN,NaN
4,Andorra,WESTERN EUROPE,71201,468,152.1,0.00,6.60,4.05,19000.0,100.0,497.2,2.22,0.00,97.78,3.0,8.71,6.25,NaN,NaN,NaN


# Transform adult DataFrame

In [4]:
# Create a filtered dataframe from specific columns
adult_cols = ["age", "education", "sex", "hours.per.week", "native.country"] 
adult_transformed = adult_df[adult_cols].copy()
adult_transformed.head()

# # Rename the column headers
adult_transformed = adult_transformed.rename(columns={"hours.per.week": "hours_per_week",
                                                     "native.country": "native_country"})
adult_transformed.head()

,age,education,sex,hours_per_week,native_country
0,90,HS-grad,Female,40,United-States
1,82,HS-grad,Female,18,United-States
2,66,Some-college,Female,40,United-States
3,54,7th-8th,Female,40,United-States
4,41,Some-college,Female,40,United-States


In [5]:
# Exclude rows with a '?' and rows from "Other-US(Guam-USVI-etc)", "South", "Yugoslavia", and "Scotland" 
# from the native_country column.
clean_adult_one = adult_transformed.loc[adult_transformed['native_country'] != "?"]
clean_adult = clean_adult_one.loc[clean_adult_one['native_country'] != "Outlying-US(Guam-USVI-etc)"]
clean_adult = clean_adult.loc[clean_adult['native_country'] != "South"]
clean_adult = clean_adult.loc[clean_adult['native_country'] != "Yugoslavia"]
clean_adult = clean_adult.loc[clean_adult['native_country'] != "Scotland"]
clean_adult.head()

,age,education,sex,hours_per_week,native_country
0,90,HS-grad,Female,40,United-States
1,82,HS-grad,Female,18,United-States
2,66,Some-college,Female,40,United-States
3,54,7th-8th,Female,40,United-States
4,41,Some-college,Female,40,United-States


In [6]:
# Review the clean_adult 'native_country' data
clean_adult['native_country'].value_counts()

United-States         29170
Mexico                  643
Philippines             198
Germany                 137
Canada                  121
Puerto-Rico             114
El-Salvador             106
India                   100
Cuba                     95
England                  90
Jamaica                  81
China                    75
Italy                    73
Dominican-Republic       70
Vietnam                  67
Guatemala                64
Japan                    62
Poland                   60
Columbia                 59
Taiwan                   51
Haiti                    44
Iran                     43
Portugal                 37
Nicaragua                34
Peru                     31
Greece                   29
France                   29
Ecuador                  28
Ireland                  24
Hong                     20
Cambodia                 19
Trinadad&Tobago          19
Thailand                 18
Laos                     18
Honduras                 13
Hungary             

In [7]:
# Rename country names with hyphens and other countries to match the country df.
clean_adult['native_country'].replace(['United-States', 'El-Salvador', 'Puerto-Rico', 'Dominican-Republic', 'Holand-Netherlands', 'Trinadad&Tobago', 'England', 'Hong', 'Columbia'], ['United States', 'El Salvador', 'Puerto Rico', 'Dominican Republic', 'Netherlands', 'Trinidad & Tobago', 'United Kingdom', 'Hong Kong', 'Colombia'], inplace=True)
clean_adult['native_country'].value_counts()

United States         29170
Mexico                  643
Philippines             198
Germany                 137
Canada                  121
Puerto Rico             114
El Salvador             106
India                   100
Cuba                     95
United Kingdom           90
Jamaica                  81
China                    75
Italy                    73
Dominican Republic       70
Vietnam                  67
Guatemala                64
Japan                    62
Poland                   60
Colombia                 59
Taiwan                   51
Haiti                    44
Iran                     43
Portugal                 37
Nicaragua                34
Peru                     31
Greece                   29
France                   29
Ecuador                  28
Ireland                  24
Hong Kong                20
Trinidad & Tobago        19
Cambodia                 19
Thailand                 18
Laos                     18
Honduras                 13
Hungary             

# Transform country DataFrame

In [8]:
# Create a filtered dataframe from specific columns
country_cols = ["Country", "Region", "Population", "GDP ($ per capita)", "Phones (per 1000)"] 
country_transformed = country_df[country_cols].copy()

# Rename the column headers
country_transformed = country_transformed.rename(columns={"Country": "native_country",
                                                         "Region": "region",
                                                         "Population": "population",
                                                         "GDP ($ per capita)": "gdp",
                                                         "Phones (per 1000)": "phones_per_1000"})

# Remove the extra space after each country name in the 'native_country' column
country_transformed['native_country'] = country_transformed['native_country'].str.rstrip()

# country_transformed['native_country'].unique()

In [9]:
# Replace the blank cells in the 'gdp' column with zeros.
country_transformed['gdp'].replace('', '0', inplace=True)
country_transformed['gdp'].value_counts()

800.0      7
700.0      6
1900.0     5
1800.0     5
600.0      4
          ..
8300.0     1
36000.0    1
2700.0     1
16900.0    1
17200.0    1
Name: gdp, Length: 130, dtype: int64

In [10]:
# Replace the blank cells in the 'phones_per_1000' column with zeros.
country_transformed['phones_per_1000'].replace('', '0', inplace=True)
country_transformed['phones_per_1000'].value_counts()
country_transformed.dtypes

native_country      object
region              object
population           int64
gdp                float64
phones_per_1000    float64
dtype: object

# Create database connection

In [11]:
# Create engine
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')
conn = engine.connect()

In [12]:
# Confirm tables
engine.table_names()

['country', 'adult']

# Load DataFrames into database

In [13]:
# Load country_df into database
#  country_transformed.to_sql(name='country', con=engine, if_exists='append', index=False)

In [14]:
# Load adult_df into database
clean_adult.to_sql(name='adult', con=engine, if_exists='append', index=False)